<a href="https://colab.research.google.com/github/dev-sampsonorson/chibuzor/blob/main/assignment-1/Assignment_1_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1: Load data into Google Colab

In [2]:
import pandas as pd
from tabulate import tabulate

url_csv = 'https://raw.githubusercontent.com/dev-sampsonorson/chibuzor/main/assignment-1/Part1.csv'

df = pd.read_csv(url_csv)

### Step 2: Explore the dataset
**Display the dataframe**

In [ ]:
# top 5, bottom 5
df

In [ ]:
# display all rows in an easy to read format
print(tabulate(df, headers = 'keys', tablefmt = 'grid'))

**Columns & data types in DataFrame**

In [5]:
df.dtypes

ID                           int64
Insurance Type              object
Income of Policy Holder      int64
Marital Status              object
Num Claimants                int64
Injury Type                 object
Overnight Hospital Stay     object
Claim Amount                 int64
Total Claimed                int64
Num Claims                   int64
Num Soft Tissue            float64
% Soft Tissue              float64
Claim Amount Received        int64
Fraud Flag                   int64
dtype: object

**Display data (Series) of a single column**

In [ ]:
insurance_type_series = df['Insurance Type ']
# insurance_type_series.array
print(tabulate(pd.DataFrame(insurance_type_series.to_list(), columns = ['Insurance Type']), headers='keys', tablefmt='grid'))

**Add new column "Revenue"**

In [ ]:
df['Revenue'] = df["Num Claimants"] + df["Claim Amount Received"]
print(tabulate(pd.DataFrame(df, columns = ["Num Claimants", "Claim Amount Received", "Revenue"]), headers='keys', tablefmt='grid'))

**Select and display a specific row**

In [32]:
df.iloc[0:1, 0:5]

,ID,Insurance Type,Income of Policy Holder,Marital Status,Num Claimants
0,1,CI,0,NaN,2
